In [ ]:
import torch
import os
from PoseTransformer import PoseTransformer, CrossModalTransformer
from NoiseScheduler import NoiseScheduler
from transformers import CLIPModel, CLIPTokenizer
from utils.motion_process import recover_from_ric

MODEL_PATH = "model_saves/TT_0.75_percentdata_lr0.0001_wd0.01"
EMBEDDING_DIM = 512
POSE_FEATURES_DIM = 263
CLIP_MAX_LENGTH = 77

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load trained models
print("Loading models...")
pose_transformer = PoseTransformer(
    pose_dim=POSE_FEATURES_DIM,
    embedding_dim=EMBEDDING_DIM,
    num_heads=8,
    num_layers=6,
    dropout=0.1,
    use_decoder=False
).to(device)

text_cross_transformer = CrossModalTransformer(
    pose_dim=EMBEDDING_DIM,
    memory_dim=EMBEDDING_DIM,
    embedding_dim=EMBEDDING_DIM,
    num_heads=8,
    num_layers=6,
    use_decoder=True
).to(device)

noise_predictor = torch.nn.Linear(EMBEDDING_DIM, POSE_FEATURES_DIM).to(device)

# CLIP Model and Tokenizer
clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model checkpoint not found at: {MODEL_PATH}")
# weights only to true to stop receiving the warning
checkpoint = torch.load(MODEL_PATH, map_location=device, weights_only=True)

pose_transformer.load_state_dict(checkpoint["pose_transformer"])
text_cross_transformer.load_state_dict(checkpoint["text_cross_transformer"])
noise_predictor.load_state_dict(checkpoint["noise_predictor"])
print("Model weights loaded successfully.")

pose_transformer.eval()
text_cross_transformer.eval()
noise_predictor.eval()
clip_model.eval()

noise_scheduler = NoiseScheduler(timesteps=1000)

c:\Users\Kelvin C\.conda\envs\machine_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Loading models...
1000
Model weights loaded successfully.


In [3]:
@torch.no_grad()
def infer(text_input, seq_length=60, batch_size=1):

    # Tokenize text input and get text embeddings
    encoded_texts = clip_tokenizer(
        text_input,
        truncation=True,
        padding="max_length",
        max_length=CLIP_MAX_LENGTH,
        return_tensors="pt"
    )
    text_inputs = {
        "input_ids": encoded_texts["input_ids"].to(device),
        "attention_mask": encoded_texts["attention_mask"].to(device)
    }

    text_embeddings = clip_model.get_text_features(**text_inputs)
    
    if text_embeddings.ndim == 2:
         text_embeddings = text_embeddings.unsqueeze(1)

    # Initialize random noise as the starting pose
    pose_shape = (batch_size, seq_length, POSE_FEATURES_DIM)
    current_pose = torch.randn(pose_shape, device=device)
    pose_mask = torch.ones((batch_size, seq_length), device=device, dtype=torch.float32) # <-- Create mask

    print("Starting denoising loop...")

    # Perform reverse diffusion (denoising)
    for t in reversed(range(noise_scheduler.timesteps)):
        timesteps = torch.full((batch_size,), t, device=device, dtype=torch.long)

        # Get pose embeddings from transformer
        pose_embeddings = pose_transformer(
            current_pose,
            pose_mask=pose_mask,
            timesteps=timesteps
        )

        # Cross-attention with text embeddings
        text_conditioned_embeddings = text_cross_transformer(
            pose_embeddings,
            text_embeddings,
            pose_mask, 
            memory_mask=None
        )

        # Predict noise
        predicted_noise = noise_predictor(text_conditioned_embeddings)

        beta_t = noise_scheduler.betas[t].to(device)
        alpha_t = noise_scheduler.alphas[t].to(device)
        sqrt_one_minus_alpha_cumprod_t = noise_scheduler.sqrt_one_minus_alphas_cumprod[t].to(device)
        sqrt_alpha_t = torch.sqrt(alpha_t) 

        # Calculate the term multiplying the predicted noise
        noise_coeff = beta_t / sqrt_one_minus_alpha_cumprod_t

        # Calculate the mean of x_{t-1}
        mean_x_t_minus_1 = (1.0 / sqrt_alpha_t) * (current_pose - noise_coeff * predicted_noise)

        # Add noise (variance term) - except for the last step (t=0)
        if t > 0:
            variance = beta_t # Use beta_t for variance (sigma_t = sqrt(beta_t))
            std_dev = torch.sqrt(variance)
            noise = torch.randn_like(current_pose)
            current_pose = mean_x_t_minus_1 + std_dev * noise # Update current_pose to x_{t-1}
        else:
            current_pose = mean_x_t_minus_1

    return current_pose.cpu()


In [ ]:
# Example usage
text_prompt = "A person walking in a straight line."
generated_poses = infer(text_prompt, seq_length=60, batch_size=1)
pose_single = generated_poses[0].numpy()
print(pose_single.shape)


Starting denoising loop...
(80, 263)


In [31]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, FFMpegFileWriter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import mpl_toolkits.mplot3d.axes3d as p3

def plot_3d_motion(save_path, kinematic_tree, joints, title, figsize=(10, 10), fps=120, radius=4):
    matplotlib.use('Agg')

    title_sp = title.split(' ')
    if len(title_sp) > 10:
        title = '\n'.join([' '.join(title_sp[:10]), ' '.join(title_sp[10:])])
    def init():
        ax.set_xlim3d([-radius / 2, radius / 2])
        ax.set_ylim3d([0, radius])
        ax.set_zlim3d([0, radius])
        # print(title)
        fig.suptitle(title, fontsize=20)
        ax.grid(b=False)

    def plot_xzPlane(minx, maxx, miny, minz, maxz):
        ## Plot a plane XZ
        verts = [
            [minx, miny, minz],
            [minx, miny, maxz],
            [maxx, miny, maxz],
            [maxx, miny, minz]
        ]
        xz_plane = Poly3DCollection([verts])
        xz_plane.set_facecolor((0.5, 0.5, 0.5, 0.5))
        ax.add_collection3d(xz_plane)

    #         return ax

    # (seq_len, joints_num, 3)
    data = joints.copy().reshape(len(joints), -1, 3)
    # fig = plt.figure(figsize=figsize)
    # ax = p3.Axes3D(fig)
    fig, ax = plt.subplots(subplot_kw={'projection': '3d'}, figsize=figsize)  # Use subplots to create a 3D axis

    init()
    MINS = data.min(axis=0).min(axis=0)
    MAXS = data.max(axis=0).max(axis=0)
    colors = ['red', 'blue', 'black', 'red', 'blue',  
              'darkblue', 'darkblue', 'darkblue', 'darkblue', 'darkblue',
             'darkred', 'darkred','darkred','darkred','darkred']
    frame_number = data.shape[0]
    #     print(data.shape)

    height_offset = MINS[1]
    data[:, :, 1] -= height_offset
    trajec = data[:, 0, [0, 2]]
    
    data[..., 0] -= data[:, 0:1, 0]
    data[..., 2] -= data[:, 0:1, 2]

    #     print(trajec.shape)

    def update(index):
        #         print(index)
        # ax.lines = []
        # ax.collections = []
        ax.cla()
        ax.view_init(elev=120, azim=-90)
        ax.dist = 7.5
        #         ax =
        plot_xzPlane(MINS[0]-trajec[index, 0], MAXS[0]-trajec[index, 0], 0, MINS[2]-trajec[index, 1], MAXS[2]-trajec[index, 1])
#         ax.scatter(data[index, :22, 0], data[index, :22, 1], data[index, :22, 2], color='black', s=3)
        
        if index > 1:
            ax.plot3D(trajec[:index, 0]-trajec[index, 0], np.zeros_like(trajec[:index, 0]), trajec[:index, 1]-trajec[index, 1], linewidth=1.0,
                      color='blue')
        #             ax = plot_xzPlane(ax, MINS[0], MAXS[0], 0, MINS[2], MAXS[2])
        
        
        for i, (chain, color) in enumerate(zip(kinematic_tree, colors)):
#             print(color)
            if i < 5:
                linewidth = 4.0
            else:
                linewidth = 2.0
            ax.plot3D(data[index, chain, 0], data[index, chain, 1], data[index, chain, 2], linewidth=linewidth, color=color)
        #         print(trajec[:index, 0].shape)

        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])

    ani = FuncAnimation(fig, update, frames=frame_number, interval=1000/fps, repeat=False)

    ani.save(save_path, fps=fps)
    plt.close()


In [32]:
import matplotlib.pyplot as plt
import numpy as np
from utils.utils import *

%matplotlib inline

joints_num = 22
kinematic_chain = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]
# print(example_data_ml3d[0:5])

joint = recover_from_ric(torch.from_numpy(pose_single).float(), joints_num).numpy()
joint = motion_temporal_filter(joint, sigma=1)

trajectory = joint[:, 0, :]
x = trajectory[:, 0]
z = trajectory[:, 2]
n = len(x)
plot_3d_motion("output/test_ani.mp4", kinematic_chain, joint, title="Testing!", fps=10)
# colors = plt.cm.viridis(np.linspace(0, 1, n))
# plt.scatter(x, z, c=colors, s=5)
# plt.plot(x, z, linewidth=1)  # Plot X-Z movement
# plt.xlabel('X')
# plt.ylabel('Z')
# plt.title('Root Joint Trajectory')
# plt.show()